In [1]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import Angle
import matplotlib.pyplot as plt
from scipy.stats import scoreatpercentile
from scipy import ndimage,signal
import glob
import os
import sys

from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker

from ppxf.ppxf import ppxf, rebin
import ppxf.ppxf_util as util
from ppxf import miles_util as lib

import threading
from argparse import ArgumentParser

c = 299792.458 # speed of light in km/s

In [2]:
#### VARIABLES ####
root_in            = "/Users/00094926/repos/A3DRA/project_caro/simspin_files/cubes/"
root_out           = "/Users/00094926/repos/A3DRA/project_caro/simspin_files/cubes/ppxf_output/"
template_loc       = "/Users/00094926/repos/A3DRA/project_caro/EMILES_PADOVA_CH/Ech1.30*.fits"
file_name_list     = ["disk_EMILES_spectral_fwhm_lowz",
                      "disk_EMILES_spectral_fwhm_highz",
                      "disk_BC03hr_spectral_fwhm_lowz",
                      "disk_BC03hr_spectral_fwhm_highz",
                      "disk_EMILES_spectral_fwhm_lowz_blur",
                      "disk_EMILES_spectral_fwhm_highz_blur",
                      "disk_BC03hr_spectral_fwhm_lowz_blur",
                      "disk_BC03hr_spectral_fwhm_highz_blur"]
folder_list        = ["apr26_2023/fwhm_lowz/disk_EMILES/",
                      "apr26_2023/fwhm_highz/disk_EMILES/",
                      "apr26_2023/fwhm_lowz/disk_BC03hr/",
                      "apr26_2023/fwhm_highz/disk_BC03hr/",
                      "apr26_2023/fwhm_lowz_blur/disk_EMILES/",
                      "apr26_2023/fwhm_highz_blur/disk_EMILES/",
                      "apr26_2023/fwhm_lowz_blur/disk_BC03hr/",
                      "apr26_2023/fwhm_highz_blur/disk_BC03hr/"]
redshift           = [0.0144, 0.3, 0.0144, 0.3, 0.0144, 0.3, 0.0144, 0.3]
telescope_FWHM_list= [3.61, 3.61, 4.56, 4.56, 3.61, 3.61, 4.56, 4.56]
template_FWHM_list = [2.51, 2.51, 3, 3, 2.51, 2.51, 3, 3]
n_mom              = 4
plot               = True


In [3]:
for i in range(0,8):
    
    folder = folder_list[i]
    template_FWHM = template_FWHM_list[i]
    file_name = file_name_list[i]
    telescope_FWHM = telescope_FWHM_list[i]
    FWHM_temp = 2.51
    z = redshift[i]
        
    #### COMPUTATION based on variables ####
    if telescope_FWHM <= template_FWHM:
        FWHM_gal = (template_FWHM * (1 + z)) # Angstroms (resolution for this "observed" template)
    else:
        FWHM_gal = telescope_FWHM

    file_root = f"{root_in}{folder}"
    ppxf_out_root = f"{root_out}{folder}"
    #########################################

    inputFileExists = os.path.exists(f"{file_root}{file_name}.FITS")
    outputDirectoryExists = os.path.exists(ppxf_out_root)

    if not inputFileExists or not outputDirectoryExists:
        print("\nError - check you file names before proceading.")

        if not inputFileExists:
            print(f"Input FITS file (",f"{file_root}{file_name}.FITS", ") does not exist. \n")
            sys.exit()
        else:
            print("Output directory (",f"{ppxf_out_root}",") for pPXF files does not exist. \n")
            sys.exit()

    # Check n_mom is 2 or 4

    if n_mom != 2. and n_mom != 4.:
        print("Number of requested velocity moments must be 2 OR 4.")
        sys.exit()

    # Make a new pPXF fit folder at the location of ppxf_out_root for fitting plots
    try:
      os.makedirs(f"{ppxf_out_root}/pPXF_fits", exist_ok=True)
    except OSError as error:
      print("Invalid path name for output pPXF files. \n")
      sys.exit()
        
    #---------------------------------#
    # Step 1: Read in the observation #
    #---------------------------------#

    hdu = fits.open(f"{file_root}{file_name}.FITS")

    cube     = hdu["DATA"].data
    head_obs = hdu["DATA"].header
    var      = hdu["STAT"].data*1e40
    
    npart    = hdu["NPART"].data
    npart_flat = npart.reshape(-1)
    npart_flat[npart_flat == 0] = None

    hdu.close()
    
    print(f"Reading spectral cube of size {cube.shape}.")

    # Reshape the cube into an array
    npix = cube.shape[0]
    nspec = cube.shape[1] * cube.shape[2]
    spectrum = cube.reshape(npix, -1) # create an array of spectra [npix, (sbin_x * sbin_y)]
    noise_flat = var.reshape(npix, -1) 

    print(f"Re-shaping cube into array of shape {spectrum.shape}, with {spectrum.shape[0]} wavelengths and {spectrum.shape[1]} pixels.")
    print(f"Re-shaping noise into array of shape {noise_flat.shape}, with {noise_flat.shape[0]} wavelengths and {noise_flat.shape[1]} pixels.")

    # Begin by pulling out the wavelength axis of the cube -------------------------------------------------------------
    # These properties are necessary for logarithmically re-binning the spectra before
    # fitting with pPXF.
    wave_axis = head_obs['CRVAL3'] + (np.arange(0., (head_obs['NAXIS3']))*head_obs['CDELT3'])
    wave_range = head_obs["CRVAL3"] + np.array([0., head_obs["CDELT3"]*(head_obs["NAXIS3"]-1)])
    velscale = c * np.diff(np.log(wave_axis[-2:])) # Smallest velocity step
    velscale = velscale[0]

    print(f"The velocity scale of the `observed` spectrum is {velscale} km/s per pixel.")

    # We also need to bring the observed spectrum back to the rest-frame ------------------------------------
    if z >= 0.0:
        wave_range = wave_range / (1 + z)
        FWHM_gal = FWHM_gal / (1 + z)
        z = 0
        print(f"The wavelength range of the 'observed' galaxy (returned to rest-frame) is {wave_range} Angstrom.")
        print(f"The FWHM at this restframe is {FWHM_gal}.")

    else:
        print(f"The wavelength range of the 'observed' galaxy is {wave_range} Angstrom.")
        print(f"The FWHM of the observation is {FWHM_gal}.")

    #---------------------------------#
    # Step 2: Read in the templates   #
    #---------------------------------#
    # Next, load the templates necessary to perform the fit with pPXF

    # Using pPXF function to read in the templates located at `template_loc`, this
    # function reads each template spectrum into an array sorted by age and metallicity.
    # If the FWHM_gal is larger than the FWHM_temp, this function will also convolve
    # the templates down to the same resolution as the observations.

    print(f"The FWHM of the templates is {FWHM_temp}.")
    if FWHM_gal <= FWHM_temp:
        FWHM_gal = None
        FWHM_temp = None
        print(f"No convolution can be made. Setting FWHM_gal, FWHM_temp = None.")

    miles = lib.miles(pathname = template_loc, velscale = velscale, FWHM_gal = FWHM_gal, FWHM_tem = FWHM_temp, norm_range=[5000, 5950])

    templates = miles.templates
    stars_templates = templates.reshape(templates.shape[0], -1)
    stars_templates /= np.median(stars_templates) # Normalizes stellar templates by a scalar

    log_lam_temp = miles.ln_lam_temp
    #reg_dim = miles.templates.shape[1:]
    #regul_err = 0.013  # Desired regularization error

    mask = (np.exp(log_lam_temp) > (np.min(wave_range)-500)) & (np.exp(log_lam_temp) < (np.max(wave_range)+500))

    log_lam_temp = log_lam_temp[mask]
    stars_templates = stars_templates[mask]

    wave_range_temp =[np.exp(np.min(log_lam_temp)), np.exp(np.max(log_lam_temp))]

    print(f"The wavelength range of the template spectra is {wave_range_temp} Angstrom.")
    print(f"Array of templates containing shape {stars_templates.shape}.")

    #----------------------------------------#
    # Step 3: Fitting the observed spectra   #
    #----------------------------------------#

    # Preparing the galaxy observations for input into pPXF
    galaxy, log_lam_galaxy, velscale = util.log_rebin(wave_range, spectrum[:,11], velscale=velscale)
    galaxy /= np.median(galaxy)

    dv = (log_lam_temp[0] - log_lam_galaxy[0])*c
    vel = c*np.log(1 + z)
    start = [vel, 100.] # starting guess for kinematic measurements

    print(f"The starting guess velocity, as according to Cappellari 2017 [V = c*log(1 + z)] is {vel} km/s.")

    # Now fitting each pixel in turn and comparing to the LOS_VEL and LOS_DISP values ----------------------------------
    if n_mom == 2:
        ppxf_velocity    = np.empty_like(npart_flat)
        ppxf_dispersion  = np.empty_like(npart_flat)
        ppxf_chi2        = np.empty_like(npart_flat)
    else:
        ppxf_velocity    = np.empty_like(npart_flat)
        ppxf_dispersion  = np.empty_like(npart_flat)
        ppxf_h3          = np.empty_like(npart_flat)
        ppxf_h4          = np.empty_like(npart_flat)
        ppxf_chi2        = np.empty_like(npart_flat)

    print(f"The 'observed' spectrum has a staring wavelength of {log_lam_galaxy[0]} Angstrom.")
    print(f"The template spectrum has a staring wavelength of {log_lam_temp[0]} Angstrom.")
    print(f"The difference is therefore {dv} km/s.")

    for j in range(nspec):
        if npart_flat[j] > 0:
            print(f"Fitting pixel [{j}]...")
            galaxy, log_lam_galaxy, velscale = util.log_rebin(wave_range, spectrum[:,j], velscale=velscale)
            noise_rebin, log_lam_noise, velscale_2 = util.log_rebin(wave_range, 1/np.sqrt(noise_flat[:,j]), velscale=velscale)
            noise_rebin /= np.median(galaxy)
            galaxy /= np.median(galaxy)
            
            if n_mom == 2:
                pp = ppxf(stars_templates, galaxy, noise_rebin, velscale, start,
                          vsyst=dv, lam = np.exp(log_lam_galaxy),
                          plot=False, moments=2, degree=-1, mdegree=10,  # Set plot to False when using in batch mode. change mdegree=10 for testing
                          clean=False, regul=False)
                ppxf_velocity[j]   = pp.sol[0]
                ppxf_dispersion[j] = pp.sol[1]
                ppxf_chi2[j]       = pp.chi2

                if plot:
                  pp.plot()
                  plt.savefig(f"{ppxf_out_root}/pPXF_fits/pPXF_fit_{j}.png")
                  plt.close()

            else:
                pp = ppxf(stars_templates, galaxy, noise_rebin, velscale, start,
                          vsyst=dv, lam = np.exp(log_lam_galaxy), 
                          plot=False, moments=4, degree=-1, mdegree=10,  # Set plot to False when using in batch mode. change mdegree=10 for testing
                          clean=False, regul=False)
                
                ppxf_velocity[j]   = pp.sol[0]
                ppxf_dispersion[j] = pp.sol[1]
                ppxf_h3[j]         = pp.sol[2]
                ppxf_h4[j]         = pp.sol[3]
                ppxf_chi2[j]       = pp.chi2

                if plot:
                  pp.plot()
                  plt.savefig(f"{ppxf_out_root}/pPXF_fits/pPXF_fit_{j}.png")
                  plt.close()


        else:
            if n_mom == 2:
                print(f"Pixel [{j}] is empty, proceading to next one...")
                ppxf_velocity[j]   = np.nan
                ppxf_dispersion[j] = np.nan
                ppxf_chi2[j]       = np.nan

            else:
                print(f"Pixel [{j}] is empty, proceading to next one...")
                ppxf_velocity[j]   = np.nan
                ppxf_dispersion[j] = np.nan
                ppxf_h3[j]         = np.nan
                ppxf_h4[j]         = np.nan
                ppxf_chi2[j]       = np.nan

    # --------------------------------- #
    # Step 4: Reshaping the fit results #
    #         back to images            #
    # --------------------------------- #

    fit_velocity   = ppxf_velocity.reshape([cube.shape[1], cube.shape[2]])
    fit_dispersion = ppxf_dispersion.reshape([cube.shape[1], cube.shape[2]])
    fit_chi2       = ppxf_chi2.reshape([cube.shape[1], cube.shape[2]])

    np.save(f"{ppxf_out_root}{file_name}_ppxf_velocity.npy", fit_velocity)
    np.save(f"{ppxf_out_root}{file_name}_ppxf_dispersion.npy", fit_dispersion)
    np.save(f"{ppxf_out_root}{file_name}_ppxf_chi2.npy", fit_chi2)

    if n_mom == 4:

        fit_h3 = ppxf_h3.reshape([cube.shape[1], cube.shape[2]])
        fit_h4 = ppxf_h4.reshape([cube.shape[1], cube.shape[2]])
        np.save(f"{ppxf_out_root}{file_name}_ppxf_h3.npy", fit_h3)
        np.save(f"{ppxf_out_root}{file_name}_ppxf_h4.npy", fit_h4)
        
    print(f"Done with the {i} iteration.")
    del cube, galaxy, noise_rebin, var, head_obs


Reading spectral cube of size (1924, 30, 30).
Re-shaping cube into array of shape (1924, 900), with 1924 wavelengths and 900 pixels.
Re-shaping noise into array of shape (1924, 900), with 1924 wavelengths and 900 pixels.
The velocity scale of the `observed` spectrum is 54.228990358951044 km/s per pixel.
The wavelength range of the 'observed' galaxy (returned to rest-frame) is [3696.76656151 5668.29652997] Angstrom.
The FWHM at this restframe is 3.5587539432176656.
The FWHM of the templates is 2.51.
The wavelength range of the template spectra is [3196.81691845587, 6167.683415503795] Angstrom.
Array of templates containing shape (3634, 258).
The starting guess velocity, as according to Cappellari 2017 [V = c*log(1 + z)] is 0.0 km/s.
The 'observed' spectrum has a staring wavelength of 8.215165582202367 Angstrom.
The template spectrum has a staring wavelength of 8.06991088075 Angstrom.
The difference is therefore -43546.26398446149 km/s.
Pixel [0] is empty, proceading to next one...
Pixel

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -26        98     0.001    -0.029
chi2/DOF: 1.657; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 10; Func calls: 154; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 8/258
Fitting pixel [51]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -31       109    -0.015    -0.089
chi2/DOF: 1.974; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 5; Func calls: 77; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 9/258
Pixel [52] is empty, proceading to next one...
Pixel [53] is empty, proceading to next one...
Pixel [54] is empty, proceading to next one...
Pixel [55] is empty, proceading to next one...
Pixel [56] is empty, proceading to next one...
Pixel [57] is empty, proceading to next one...
Pixel [58] is empty, proceading to next one...
Pixel [59] is empty, proceading to next one...
Pixel [60] is empty, proceading to next one...
Pixel [61] 

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:         1       106     0.002    -0.035
chi2/DOF: 1.319; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 6; Func calls: 92; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 11/258
Fitting pixel [102]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        12        88    -0.079     0.130
chi2/DOF: 2.157; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 139; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 9/258
Fitting pixel [103]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -4        99     0.003    -0.016
chi2/DOF: 1.703; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 12; Func calls: 185; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/258
Fitting pixel [104]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -8       104     0.017     0.014
chi2/DOF: 1.

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:         5        67    -0.206     0.300
chi2/DOF: 3.703; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 23; Func calls: 353; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 8/258
Fitting pixel [138]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -19        91    -0.026     0.055
chi2/DOF: 3.228; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 110; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 9/258
Fitting pixel [139]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -23       106     0.013    -0.021
chi2/DOF: 1.235; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 6; Func calls: 92; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 11/258
Fitting pixel [140]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -25       111    -0.022    -0.041
chi2/DOF: 1.

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -25       105    -0.036    -0.024
chi2/DOF: 1.475; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 6; Func calls: 92; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 9/258
Fitting pixel [172]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -41        99     0.027     0.031
chi2/DOF: 2.801; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 111; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [173]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -55       104     0.022    -0.039
chi2/DOF: 4.792; DOF: 2350; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 125; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 10/258
Fitting pixel [174]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -34        96    -0.011    -0.023
chi2/DOF: 1.1

...

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -69       142    -0.060     0.215
chi2/DOF: 1049.; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 128; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/258
Fitting pixel [354]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -94        94     0.209    -0.095
chi2/DOF: 191.0; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 112; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [355]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -49         2     0.000     0.000
chi2/DOF: 298.8; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 15; Func calls: 233; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/258
Fitting pixel [356]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -113        55     0.300     0.300
chi2/DOF: 195.5

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -1       122     0.300    -0.136
chi2/DOF: 310.5; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 10; Func calls: 154; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 5/258
Fitting pixel [389]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        -2        82     0.300     0.300
chi2/DOF: 110.6; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 12; Func calls: 187; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 4/258
Fitting pixel [390]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        36        81    -0.031     0.187
chi2/DOF: 339.2; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 109; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [391]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        65       349     0.043    -0.042
chi2/DOF: 560.

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -27        92    -0.000     0.001
chi2/DOF: 266.0; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 11; Func calls: 171; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 5/258
Fitting pixel [422]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -66       127     0.020     0.104
chi2/DOF: 393.0; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 137; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/258
Fitting pixel [423]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -35       101     0.047    -0.115
chi2/DOF: 241.8; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 125; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [424]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -45       125    -0.018     0.055
chi2/DOF: 136.2

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -26        38     0.230     0.279
chi2/DOF: 88.76; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 109; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/258
Fitting pixel [463]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        24        99     0.013     0.010
chi2/DOF: 211.6; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 141; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [464]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        33       113    -0.141    -0.131
chi2/DOF: 95.32; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 10; Func calls: 154; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Fitting pixel [465]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        19       109    -0.085     0.007
chi2/DOF: 63.71

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -78        59     0.001    -0.001
chi2/DOF: 45.81; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 142; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/258
Pixel [499] is empty, proceading to next one...
Pixel [500] is empty, proceading to next one...
Pixel [501] is empty, proceading to next one...
Pixel [502] is empty, proceading to next one...
Pixel [503] is empty, proceading to next one...
Pixel [504] is empty, proceading to next one...
Pixel [505] is empty, proceading to next one...
Pixel [506] is empty, proceading to next one...
Pixel [507] is empty, proceading to next one...
Pixel [508] is empty, proceading to next one...
Fitting pixel [509]...
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        95       165    -0.030     0.300
chi2/DOF: 50.38; DOF: 805; degree = -1; mdegree = 10
method = capfit; Jac calls: 13; Func calls: 201; Status: 2
linear_metho